In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

np.random.seed(42)
random.seed(42)

#############################
# 1. PERFORMANCE_TB 생성
#############################

n_performance = 100000  # 예시: 100개 공연

# performance_id: 1001부터 순차적 부여
performance_ids = np.arange(1001, 1001+n_performance)

# performance_name: "공연_1001", "공연_1002", ... 
performance_names = [f"공연_{pid}" for pid in performance_ids]

# genre 분포 (가중치: 평균값 사용)
genres = ["연극", "뮤지컬", "무용", "대중무용", "서양음악", "한국음악", "대중음악", "복합", "서커스/마술"]
genre_weights = np.array([17.5, 30, 7.5, 7.5, 7.5, 7.5, 17.5, 4, 2])
genre_probs = genre_weights / genre_weights.sum()
performance_genres = np.random.choice(genres, size=n_performance, p=genre_probs)

# region 분포
regions = ["서울특별시", "경기도", "부산광역시", "대구광역시", "인천광역시", 
           "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", 
           "강원특별자치도", "충청북도", "충청남도", "전라북도", "전라남도", 
           "경상북도", "경상남도", "제주특별자치도"]
region_weights = np.array([45, 17.5, 7.5, 4, 4, 3.5, 3.5, 2, 1.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 2])
region_probs = region_weights / region_weights.sum()
performance_regions = np.random.choice(regions, size=n_performance, p=region_probs)

# start_date: 공연 시작일 - 랜덤하게 2025-05-01부터 2025-07-01 사이 선택
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=np.random.randint(0, delta.days+1))

start_base = datetime(2025, 5, 1)
end_base = datetime(2025, 7, 1)
start_dates = [random_date(start_base, end_base) for _ in range(n_performance)]
# end_date: start_date로부터 1일~60일 후
end_dates = [sd + timedelta(days=np.random.randint(1, 61)) for sd in start_dates]

# capacity: 각 공연장 좌석 수
# 범위와 가중치 (범위: (min, max))
cap_ranges = [ (0, 0), (1, 299), (300, 499), (500, 999), (1000, 4999), (5000, 9999), (10000, 999999) ]
cap_weights = np.array([5, 45, 12.5, 12.5, 17.5, 4, 1.5])
cap_probs = cap_weights / cap_weights.sum()

def sample_range(ranges, probs):
    idx = np.random.choice(len(ranges), p=probs)
    rmin, rmax = ranges[idx]
    if rmin == rmax:
        return rmin
    # np.random.randint 대신 random.randint 사용 (Python 내장 함수)
    return random.randint(rmin, rmax)


capacities = [sample_range(cap_ranges, cap_probs) for _ in range(n_performance)]

# ticket_price (원)
# 범위와 가중치
price_ranges = [ (0, 0), (1, 10000), (10001, 30000), (30001, 50000), (50001, 100000), (100001, 300000) ]
price_weights = np.array([2, 7.5, 25, 30, 22.5, 7.5])
price_probs = price_weights / price_weights.sum()
ticket_prices = [sample_range(price_ranges, price_probs) for _ in range(n_performance)]

# production_cost (제작비, 단위: 원)
prod_ranges = [ (0, 10000000), (10000000, 50000000), (50000000, 100000000), (100000000, 10000000000),
                (10000000000, 50000000000), (50000000000, 1000000000000) ]
prod_weights = np.array([7.5, 12.5, 25, 30, 7.5, 3])
prod_probs = prod_weights / prod_weights.sum()
production_costs = [sample_range(prod_ranges, prod_probs) for _ in range(n_performance)]

# marketing_budget (원)
market_ranges = [ (0, 5000000), (5000000, 10000000), (10000000, 50000000), (50000000, 100000000), (100000000, 500000000) ]
market_weights = np.array([25, 30, 22.5, 12.5, 7.5])
market_probs = market_weights / market_weights.sum()
marketing_budgets = [sample_range(market_ranges, market_probs) for _ in range(n_performance)]

# star_power (0~5)
# 그룹: 0, 1, 2, 3, 4, 5
star_values = [0, 1, 2, 3, 4, 5]
star_weights = np.array([25, 15, 15, 11.25, 11.25, 12.5])
star_probs = star_weights / star_weights.sum()
star_powers = np.random.choice(star_values, size=n_performance, p=star_probs)

# sns_mention_count
sns_ranges = [ (0, 10), (11, 50), (51, 100), (101, 500), (501, 1000), (1001, 5000) ]
sns_weights = np.array([25, 35, 17.5, 12.5, 4, 2])
sns_probs = sns_weights / sns_weights.sum()
sns_mention_counts = [sample_range(sns_ranges, sns_probs) for _ in range(n_performance)]

# 데이터프레임 생성
df_performance = pd.DataFrame({
    "performance_id": performance_ids,
    "performance_name": performance_names,
    "genre": performance_genres,
    "region": performance_regions,
    "start_date": start_dates,
    "end_date": end_dates,
    "capacity": capacities,
    "ticket_price": ticket_prices,
    "production_cost": production_costs,
    "marketing_budget": marketing_budgets,
    "star_power": star_powers,
    "sns_mention_count": sns_mention_counts
})

# CSV 저장
df_performance.to_csv("performance_tb.csv", index=False, encoding='utf-8-sig')
print("performance_tb.csv 생성 완료!")

#############################
# 2. SALES_TB 생성
#############################

# 예를 들어, 각 공연별로 30일간의 판매 데이터를 생성 (총 n_performance * 30 행)
n_days = 30
sales_records = []
for pid, cap, tprice in zip(df_performance["performance_id"], df_performance["capacity"], df_performance["ticket_price"]):
    # 시작일과 종료일 범위 내에서 n_days 랜덤 날짜 생성 (단, 공연 시작일부터 종료일까지)
    # 공연별 기간: start_date ~ end_date
    start = df_performance.loc[df_performance["performance_id"]==pid, "start_date"].iloc[0]
    end = df_performance.loc[df_performance["performance_id"]==pid, "end_date"].iloc[0]
    if start > end:
        start, end = end, start
    period = (end - start).days
    if period < 1:
        period = 1
    dates = [start + timedelta(days=np.random.randint(0, period+1)) for _ in range(n_days)]
    for d in dates:
        # 누적 판매: 0 ~ capacity (단, capacity가 0이면 0)
        acc_sales = np.random.randint(0, cap+1) if cap > 0 else 0
        # 일별 판매: 0 ~ (acc_sales//n_days + 1)
        daily_sales = np.random.randint(0, (acc_sales // n_days) + 2)
        # booking_rate: (daily_sales/capacity)*100, 처리 (capacity가 0이면 0)
        booking_rate = (daily_sales/cap*100) if cap > 0 else 0
        # price_avg: ticket_price 주변 (±10%)
        price_avg = int(tprice * np.random.uniform(0.9, 1.1))
        # ad_exposure: 임의 정수 100000 ~ 500000
        ad_exposure = np.random.randint(100000, 500001)
        # sns_mention_daily: 0 ~ 100
        sns_daily = np.random.randint(0, 101)
        # promo_event_flag: 15% 확률로 True
        promo_event_flag = np.random.rand() < 0.15
        
        sales_records.append({
            "performance_id": pid,
            "date": d,
            "accumulated_sales": acc_sales,
            "daily_sales": daily_sales,
            "booking_rate": round(booking_rate, 2),
            "price_avg": price_avg,
            "ad_exposure": ad_exposure,
            "sns_mention_daily": sns_daily,
            "promo_event_flag": promo_event_flag
        })

df_sales = pd.DataFrame(sales_records)
df_sales.to_csv("sales_tb.csv", index=False, encoding='utf-8-sig')
print("sales_tb.csv 생성 완료!")

#############################
# 3. AUDIENCE_TB 생성
#############################

n_audience = 1000000  # 예시: 1000명 관객

# user_id: 2001부터 부여
audience_ids = np.arange(2001, 2001+n_audience)

# age 그룹 분포
age_groups = {
    "10대": (10, 19, 4),      # 확률: 4%
    "20대": (20, 29, 30),     # 30%
    "30대": (30, 39, 37.5),   # 37.5%
    "40대": (40, 49, 17.5),   # 17.5%
    "50대": (50, 59, 7.5),    # 7.5%
    "60대 이상": (60, 80, 3.5)  # 3.5%
}
age_labels = list(age_groups.keys())
age_weights = np.array([age_groups[g][2] for g in age_labels])
age_probs = age_weights / age_weights.sum()
audience_ages = []
for _ in range(n_audience):
    group = np.random.choice(age_labels, p=age_probs)
    low, high, _ = age_groups[group]
    age = np.random.randint(low, high+1)
    audience_ages.append(age)

# gender 분포
genders = ["F", "M", "기타"]
gender_probs = np.array([62.5, 32.5, 5]) / 100  # 여성 약 62.5%, 남성 32.5%, 기타 5%
audience_genders = np.random.choice(genders, size=n_audience, p=gender_probs)

# region 분포 (audience)
audience_regions = ["서울특별시", "경기도", "인천광역시", "부산광역시", "대구광역시", 
                    "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", 
                    "강원특별자치도", "충청북도", "충청남도", "전라북도", "전라남도", 
                    "경상북도", "경상남도", "제주특별자치도"]
audience_region_weights = np.array([40, 20, 7.5, 7.5, 3.5, 3.5, 3.5, 2, 1.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 2])
audience_region_probs = audience_region_weights / audience_region_weights.sum()
audience_regions_choice = np.random.choice(audience_regions, size=n_audience, p=audience_region_probs)

# last_booking: 관객의 최근 예매일 – 2024년부터 2025년 중 랜덤 생성 (분포 반영은 아래와 같이 그룹별 가중치 적용)
last_booking_groups = {
    "최근 1개월 이내": (datetime(2025, 4, 1), datetime(2025, 5, 1), 25),   # 25%
    "1~3개월 전": (datetime(2025, 1, 1), datetime(2025, 4, 1), 27.5),       # 27.5%
    "3~6개월 전": (datetime(2024, 10, 1), datetime(2025, 1, 1), 22.5),      # 22.5%
    "6개월~1년 전": (datetime(2024, 5, 1), datetime(2024, 10, 1), 17.5),      # 17.5%
    "1년 이상 전": (datetime(2020, 1, 1), datetime(2024, 5, 1), 7.5)           # 7.5%
}
lb_labels = list(last_booking_groups.keys())
lb_weights = np.array([last_booking_groups[g][2] for g in lb_labels])
lb_probs = lb_weights / lb_weights.sum()

audience_last_bookings = []
for _ in range(n_audience):
    group = np.random.choice(lb_labels, p=lb_probs)
    lb_start, lb_end, _ = last_booking_groups[group]
    delta_days = (lb_end - lb_start).days
    random_days = np.random.randint(0, delta_days+1)
    audience_last_bookings.append(lb_start + timedelta(days=random_days))

# booking_count 분포
# 그룹별: "1회": 7.5%, "2~3회": 35%, "4~10회": 27.5%, "11~20회": 12.5%, "21회 이상": 7.5%
bc_groups = {
    "1회": (1, 1, 7.5),
    "2~3회": (2, 3, 35),
    "4~10회": (4, 10, 27.5),
    "11~20회": (11, 20, 12.5),
    "21회 이상": (21, 50, 7.5)  # 상한은 임의로 50회로 설정
}
bc_labels = list(bc_groups.keys())
bc_weights = np.array([bc_groups[g][2] for g in bc_labels])
bc_probs = bc_weights / bc_weights.sum()
audience_booking_counts = []
for _ in range(n_audience):
    group = np.random.choice(bc_labels, p=bc_probs)
    low, high, _ = bc_groups[group]
    count = np.random.randint(low, high+1)
    audience_booking_counts.append(count)

# total_amount 분포 (누적 예매 금액, 원)
ta_ranges = [ (0, 0), (1, 9999), (10000, 49999), (50000, 99999), (100000, 299999), (300000, 499999), (500000, 1000000) ]
ta_weights = np.array([4, 7.5, 30, 25, 17.5, 7.5, 4])
ta_probs = ta_weights / ta_weights.sum()
audience_total_amounts = [sample_range(ta_ranges, ta_probs) for _ in range(n_audience)]

# preferred_genre (선호 장르) - 동일한 분포 as performance_tb genre
audience_preferred_genres = np.random.choice(genres, size=n_audience, p=genre_probs)

df_audience = pd.DataFrame({
    "user_id": audience_ids,
    "age": audience_ages,
    "gender": audience_genders,
    "region": audience_regions_choice,
    "last_booking": audience_last_bookings,
    "booking_count": audience_booking_counts,
    "total_amount": audience_total_amounts,
    "preferred_genre": audience_preferred_genres
})

df_audience.to_csv("audience_tb.csv", index=False, encoding='utf-8-sig')
print("audience_tb.csv 생성 완료!")

######################################
# 4. 분포 체크 (각 테이블별 주요 컬럼)
######################################

print("\n=== PERFORMANCE_TB 분포 체크 ===")
print("장르 분포:")
print(df_performance["genre"].value_counts(normalize=True) * 100)
print("\n지역 분포:")
print(df_performance["region"].value_counts(normalize=True) * 100)

print("\n=== AUDIENCE_TB 분포 체크 ===")
print("연령대 분포 (십의 자리로 그룹화):")
bins = [0, 19, 29, 39, 49, 59, 80]
labels = ["10대", "20대", "30대", "40대", "50대", "60대 이상"]
df_audience["age_group"] = pd.cut(df_audience["age"], bins=bins, labels=labels, right=True)
print(df_audience["age_group"].value_counts(normalize=True) * 100)

print("\n성별 분포:")
print(df_audience["gender"].value_counts(normalize=True) * 100)

print("\n지역 분포:")
print(df_audience["region"].value_counts(normalize=True) * 100)

print("\n선호 장르 분포:")
print(df_audience["preferred_genre"].value_counts(normalize=True) * 100)


performance_tb.csv 생성 완료!
sales_tb.csv 생성 완료!
audience_tb.csv 생성 완료!

=== PERFORMANCE_TB 분포 체크 ===
장르 분포:
genre
뮤지컬       29.617
연극        17.391
대중음악      17.385
서양음악       7.470
무용         7.448
대중무용       7.447
한국음악       7.428
복합         3.886
서커스/마술     1.928
Name: proportion, dtype: float64

지역 분포:
region
서울특별시      38.980
경기도        15.174
부산광역시       6.533
인천광역시       3.540
대구광역시       3.453
전라북도        3.108
경상남도        3.095
충청북도        3.092
충청남도        3.061
광주광역시       3.059
전라남도        3.049
경상북도        3.046
대전광역시       3.035
강원특별자치도     3.003
울산광역시       1.783
제주특별자치도     1.742
세종특별자치시     1.247
Name: proportion, dtype: float64

=== AUDIENCE_TB 분포 체크 ===
연령대 분포 (십의 자리로 그룹화):
age_group
30대       37.5238
20대       29.9921
40대       17.4677
50대        7.5199
10대        3.9971
60대 이상     3.4994
Name: proportion, dtype: float64

성별 분포:
gender
F     62.5238
M     32.5068
기타     4.9694
Name: proportion, dtype: float64

지역 분포:
region
서울특별시      34.5090
경기도        17.3941
인천광역시 